In [60]:
from pyspark.sql import SparkSession
import os
import sys
from config import API_KEY
from datetime import datetime, timedelta
import requests
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType, DateType, DoubleType, IntegerType
import pandas as pd
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, col, rank, avg, round, count, countDistinct, udf, when, lower
from pyspark.sql import functions as F
import sqlite3

In [3]:
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")
print(f"PATH: {os.environ.get('PATH')}")
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-1.8'
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")

#os.environ["PYSPARK_PYTHON"] = r"C:\Users\lucas\anaconda3\python.exe"
#os.environ["PYSPARK_DRIVER_PYTHON"] = r"C:\Users\lucas\anaconda3\python.exe"

JAVA_HOME: C:\Program Files\Java\jdk-1.8
PATH: c:\Users\lucas\anaconda3\envs\pyspark_env;C:\Users\lucas\anaconda3\envs\pyspark_env;C:\Users\lucas\anaconda3\envs\pyspark_env\Library\mingw-w64\bin;C:\Users\lucas\anaconda3\envs\pyspark_env\Library\usr\bin;C:\Users\lucas\anaconda3\envs\pyspark_env\Library\bin;C:\Users\lucas\anaconda3\envs\pyspark_env\Scripts;C:\Users\lucas\anaconda3\envs\pyspark_env\bin;C:\Users\lucas\anaconda3\condabin;C:\Users\lucas\anaconda3;C:\Program Files (x86)\Common Files\Oracle\Java\java8path;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\libnvvp;C:\Program Files\Rockwell Software\RSCommon;C

In [3]:
import findspark
findspark.init()

In [20]:
spark.stop()

In [4]:
spark = SparkSession.builder \
    .appName("weatherapp") \
    .getOrCreate()
    #.config("spark.driver.memory", "4g") \
    #.config("spark.executor.memory", "4g") \
    #.config("spark.python.worker.memory", "4g") \
    #.config("spark.driver.maxResultSize", "4g") \
    #.config("spark.pyspark.python", r"C:\Users\lucas\anaconda3\python.exe") \
    #.config("spark.pyspark.driver.python", r"C:\Users\lucas\anaconda3\python.exe") \
#spark.sparkContext.addFile(r"C:\Users\lucas\anaconda3\python.exe") 

In [ ]:
url = "http://api.weatherapi.com/v1/current.json"
#http://api.weatherapi.com/v1/history.json
api_key = API_KEY
city = 'Philadelphia'
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=90)
start_date = start_date.strftime("%Y-%m-%d")
params = {
    'key': api_key,
    'q': city
}

response = requests.get(url,params=params)
data = response.json()



In [36]:
data['current']

{'last_updated_epoch': 1732400100,
 'last_updated': '2024-11-23 17:15',
 'temp_c': 10.3,
 'temp_f': 50.5,
 'is_day': 0,
 'condition': {'text': 'Partly cloudy',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
  'code': 1003},
 'wind_mph': 13.2,
 'wind_kph': 21.2,
 'wind_degree': 292,
 'wind_dir': 'WNW',
 'pressure_mb': 1008.0,
 'pressure_in': 29.75,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 48,
 'cloud': 50,
 'feelslike_c': 7.6,
 'feelslike_f': 45.8,
 'windchill_c': 8.0,
 'windchill_f': 46.4,
 'heatindex_c': 10.7,
 'heatindex_f': 51.2,
 'dewpoint_c': 4.5,
 'dewpoint_f': 40.1,
 'vis_km': 16.0,
 'vis_miles': 9.0,
 'uv': 0.0,
 'gust_mph': 17.9,
 'gust_kph': 28.8}

In [ ]:
#history
def loop_date(start_date,end_date):
    date_list = []
    while start_date <= end_date:
        date_list.append(start_date.strftime("%Y-%m-%d"))
        start_date += timedelta(days=1)
    return date_list

def get_city_attr(cities,start_date,end_date):
    url = "http://api.weatherapi.com/v1/history.json"
    api_key = API_KEY
    date_list = loop_date(start_date,end_date)
    measure_metrics = ['avgtemp_f','maxwind_mph','totalprecip_in','condition']
    spark_values_store = []
    for city in cities:
        for date in date_list:
            params = {
                'key': api_key,
                'q': city,
                'dt': date
            }
            response = requests.get(url,params=params)
            data = response.json()
            date_store = {'date':date, 'city':city}
            for metric in measure_metrics:
                date_store[metric] = data['forecast']['forecastday'][0]['day'][metric]
            spark_values_store.append(date_store)
    return spark_values_store


In [5]:
def get_city_attr(cities,unit ='F'):
    url = "http://api.weatherapi.com/v1/current.json"
    api_key = API_KEY
    measure_metrics = ['name','temp_f','humidity','condition']
    spark_values_store = []
    for city in cities:
        params = {
            'key': api_key,
            'q': city
        }
        response = requests.get(url,params=params)
        data = response.json()
        data_store = {}
        for metric in measure_metrics:
            if metric == 'name':
                data_store[metric] = data['location'][metric]
                continue
            if metric == 'condition':
                data_store[metric] = data['current'][metric]['text']
                continue
            data_store[metric] = data['current'][metric]
        if unit == 'C':
            data_store['temp_c'] = (data_store['temp_f'] - 32) * (5/9)
            del data_store['temp_f']
        spark_values_store.append(data_store)
    return spark_values_store


In [6]:
cities = [
    "New York", "Los Angeles", "Chicago", "Houston", "Phoenix",
    "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose",
    "London", "Paris", "Berlin", "Madrid", "Rome",
    "Tokyo", "Beijing", "Mumbai", "Sydney", "Cape Town",
    "Dubai", "Singapore", "Seoul", "Moscow", "Rio de Janeiro",
    "Buenos Aires", "Cairo", "Bangkok", "Istanbul", "Toronto",
    "Mexico City", "Jakarta", "Kuala Lumpur", "Lagos", "Nairobi",
    "Johannesburg", "Melbourne", "Lima", "Bogotá", "Santiago",
    "Amsterdam", "Brussels", "Warsaw", "Vienna", "Stockholm",
    "Oslo", "Copenhagen", "Helsinki", "Prague", "Zurich"
]

In [12]:
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=90)

In [13]:
df_list = get_city_attr(cities,start_date,end_date)
    

In [7]:
weather_df = get_city_attr(cities)

In [15]:
schema = StructType([
    StructField('date', StringType(), True),
    StructField('city', StringType(), True),
    StructField('avgtemp_f', FloatType(), True),
    StructField('maxwind_mph', FloatType(), True),
    StructField('totalprecip_in', FloatType(), True)
])

In [ ]:
def clean_record(record):
    return {
        'date' : str(record['date']),
        'city' : str(record['city']),
        'avgtemp_f' : double(record['avgtemp_f']),
        'maxwind_mph' : double(record['maxwind_mph']),
        'totalprecip_in' : float(record['totalprecip_in'])
    }

In [8]:
weather_df = spark.createDataFrame(weather_df)

In [24]:
#find the city with the highest temperature
window_spec = Window.partitionBy().orderBy(col('temp_f').desc())
df_with_rank = weather_df.withColumn('rank',rank().over(window_spec))
city_with_highest_temp = df_with_rank.filter(col('rank')==1).select('name').collect()[0][0]
print(city_with_highest_temp)


Lagos


In [25]:
#compare temperature of 2 cities
def city_compare(city1,city2,df):
    if df.filter(col('name') == city1).select('temp_f').collect()[0][0] == df.filter(col('name') == city2).select('temp_f').collect()[0][0]:
        print(f'The temperatures of {city1} and {city2} are the same')
    elif df.filter(col('name') == city1).select('temp_f').collect()[0][0] > df.filter(col('name') == city2).select('temp_f').collect()[0][0]:
        diff_temp = df.filter(col('name') == city1).select('temp_f').collect()[0][0] - df.filter(col('name') == city2).select('temp_f').collect()[0][0]
        print(f'The temperature of {city1} is greater than {city2} by {diff_temp} degrees Farenheit')
    else:
        diff_temp = df.filter(col('name') == city2).select('temp_f').collect()[0][0] - df.filter(col('name') == city1).select('temp_f').collect()[0][0]
        print(f'The temperature of {city2} is greater than {city1} by {diff_temp} degrees Farenheit')

In [26]:
city_compare('Singapore','Sydney',weather_df)

The temperature of Singapore is greater than Sydney by 12.399999999999991 degrees Farenheit


In [27]:
df_with_rank.show()

+------------------+--------+--------------+------+----+
|         condition|humidity|          name|temp_f|rank|
+------------------+--------+--------------+------+----+
|             Sunny|      52|         Lagos|  85.0|   1|
|Patchy rain nearby|      64|       Jakarta|  84.4|   2|
|          Overcast|      54|        Mumbai|  81.3|   3|
|     Partly cloudy|      79|     Singapore|  80.8|   4|
|     Partly cloudy|      79|Rio De Janeiro|  80.6|   5|
|     Partly cloudy|      89|  Kuala Lumpur|  80.6|   5|
|             Clear|      65|       Bangkok|  79.0|   7|
|             Clear|      25|  Johannesburg|  75.6|   8|
|     Partly cloudy|      47|         Dubai|  75.2|   9|
|             Sunny|      33|      Santiago|  70.2|  10|
|     Partly Cloudy|      79|          Lima|  69.9|  11|
|             Clear|      83|        Sydney|  68.4|  12|
|     Partly cloudy|      88|  Buenos Aires|  68.0|  13|
|     Partly cloudy|      68|       Nairobi|  68.0|  13|
|     Partly cloudy|      60|  

In [28]:
def average_weather(cities,df):
    filtered_df = df.filter(col('name').isin(cities))
    avg_temp = filtered_df.agg(avg('temp_f').alias('avg_temp')).collect()[0]['avg_temp']
    avg_humidity = filtered_df.agg(avg('humidity').alias('avg_humidity')).collect()[0]['avg_humidity']
    print(f'The average temp of these cities is {avg_temp} and the average humidity of these cities is {avg_humidity}')

In [29]:
cities_for_temp_analysis = [
    "New York", "Los Angeles", "Tokyo", "Cairo", "Moscow",
    "Mumbai", "Sydney", "Cape Town", "Dubai", "Toronto",
    "Mexico City", "Lagos", "Bangkok", "Istanbul", "Berlin",
    "London", "Buenos Aires", "Jakarta", "Beijing", "Melbourne"
]

In [30]:
average_weather(cities_for_temp_analysis,weather_df)

The average temp of these cities is 59.84000000000001 and the average humidity of these cities is 64.15


In [ ]:
#create sqlite database
conn = sqlite3.connect('weather.db')

In [84]:
#create sqllite database
pandas_df = weather_df.toPandas()
pandas_df.to_sql("weather",conn,if_exists='replace',index=True)

50

In [85]:
#create sqllite database
conn.close()

In [86]:
conn = sqlite3.connect('weather.db')
result = conn.execute('SELECT * FROM weather').fetchall()

In [88]:
conn.close()

In [89]:
weather_df.show()

+-------------+--------+------------+------+
|    condition|humidity|        name|temp_f|
+-------------+--------+------------+------+
|     Overcast|      41|    New York|  53.1|
|Partly cloudy|      68| Los Angeles|  61.2|
|     Overcast|      67|     Chicago|  42.1|
|     Overcast|      64|     Houston|  64.4|
|Partly cloudy|      21|     Phoenix|  71.1|
|     Overcast|      61|Philadelphia|  52.0|
|        Clear|      47| San Antonio|  64.9|
|Partly cloudy|      60|   San Diego|  63.0|
|Partly cloudy|      69|      Dallas|  48.9|
|Partly cloudy|      80|    San Jose|  60.1|
|Partly cloudy|      81|      London|  43.3|
|     Overcast|      93|       Paris|  48.9|
|   Light rain|      87|      Berlin|  48.2|
|        Clear|      87|      Madrid|  41.7|
|Partly cloudy|      94|        Rome|  52.2|
|Partly cloudy|      54|       Tokyo|  53.6|
|        Sunny|      21|     Beijing|  34.5|
|     Overcast|      44|      Mumbai|  75.4|
|        Sunny|      65|      Sydney|  78.1|
|   Light 

In [31]:
agg_df = weather_df.groupby('condition').agg(
    avg('humidity').alias('average_humidity'),
    avg('temp_f').alias('average_temp')
)

In [32]:
agg_df.show()

+------------------+------------------+------------------+
|         condition|  average_humidity|      average_temp|
+------------------+------------------+------------------+
|     Partly cloudy|  66.9047619047619| 59.71904761904763|
|             Sunny|40.285714285714285| 61.99999999999999|
|        Light rain|              90.0|46.300000000000004|
|             Clear|              69.0| 53.28333333333333|
|          Overcast| 77.66666666666667| 49.26666666666667|
|              Mist|              96.5|              38.5|
|Patchy rain nearby|              64.0|              84.4|
|     Partly Cloudy|              79.0|              69.9|
+------------------+------------------+------------------+



In [33]:
high_temp_df = weather_df.filter(weather_df['temp_f'] > 60)

In [34]:
high_temp_df.show()

+------------------+--------+--------------+------+
|         condition|humidity|          name|temp_f|
+------------------+--------+--------------+------+
|             Sunny|      46|   Los Angeles|  61.0|
|             Sunny|      37|       Houston|  64.8|
|     Partly cloudy|      19|       Phoenix|  64.0|
|     Partly cloudy|      60|   San Antonio|  67.3|
|     Partly cloudy|      37|     San Diego|  66.9|
|          Overcast|      54|        Mumbai|  81.3|
|             Clear|      83|        Sydney|  68.4|
|     Partly cloudy|      73|     Cape Town|  66.7|
|     Partly cloudy|      47|         Dubai|  75.2|
|     Partly cloudy|      79|     Singapore|  80.8|
|     Partly cloudy|      79|Rio De Janeiro|  80.6|
|     Partly cloudy|      88|  Buenos Aires|  68.0|
|     Partly cloudy|      60|         Cairo|  66.7|
|             Clear|      65|       Bangkok|  79.0|
|             Sunny|      32|   Mexico City|  64.9|
|Patchy rain nearby|      64|       Jakarta|  84.4|
|     Partly

In [98]:
weather_df.select('condition','humidity').show()

+-------------+--------+
|    condition|humidity|
+-------------+--------+
|     Overcast|      41|
|Partly cloudy|      68|
|     Overcast|      67|
|     Overcast|      64|
|Partly cloudy|      21|
|     Overcast|      61|
|        Clear|      47|
|Partly cloudy|      60|
|Partly cloudy|      69|
|Partly cloudy|      80|
|Partly cloudy|      81|
|     Overcast|      93|
|   Light rain|      87|
|        Clear|      87|
|Partly cloudy|      94|
|Partly cloudy|      54|
|        Sunny|      21|
|     Overcast|      44|
|        Sunny|      65|
|   Light rain|      82|
+-------------+--------+
only showing top 20 rows



In [35]:
weather_df = weather_df.withColumnRenamed('temp_f','temperature')

In [36]:
weather_df = weather_df.withColumn('temp_c',round((col('temperature')-32) * (5/9),2))

In [ ]:
weather_df.show()

+-------------+--------+------------+-----------+------+
|    condition|humidity|        name|temperature|temp_c|
+-------------+--------+------------+-----------+------+
|     Overcast|      41|    New York|       53.1| 11.72|
|Partly cloudy|      68| Los Angeles|       61.2| 16.22|
|     Overcast|      67|     Chicago|       42.1|  5.61|
|     Overcast|      64|     Houston|       64.4|  18.0|
|Partly cloudy|      21|     Phoenix|       71.1| 21.72|
|     Overcast|      61|Philadelphia|       52.0| 11.11|
|        Clear|      47| San Antonio|       64.9| 18.28|
|Partly cloudy|      60|   San Diego|       63.0| 17.22|
|Partly cloudy|      69|      Dallas|       48.9|  9.39|
|Partly cloudy|      80|    San Jose|       60.1| 15.61|
|Partly cloudy|      81|      London|       43.3|  6.28|
|     Overcast|      93|       Paris|       48.9|  9.39|
|   Light rain|      87|      Berlin|       48.2|   9.0|
|        Clear|      87|      Madrid|       41.7|  5.39|
|Partly cloudy|      94|       

In [37]:
def get_city_attr2(cities,unit ='F'):
    url = "http://api.weatherapi.com/v1/current.json"
    api_key = API_KEY
    measure_metrics = ['name','gust_mph','pressure_mb']
    spark_values_store = []
    for city in cities:
        params = {
            'key': api_key,
            'q': city
        }
        response = requests.get(url,params=params)
        data = response.json()
        data_store = {}
        for metric in measure_metrics:
            if metric == 'name':
                data_store[metric] = data['location'][metric]
                continue
            data_store[metric] = data['current'][metric]
        if unit == 'C':
            data_store['temp_c'] = (data_store['temp_f'] - 32) * (5/9)
            del data_store['temp_f']
        spark_values_store.append(data_store)
    return spark_values_store


In [40]:
weather_df2 = get_city_attr2(cities)

In [41]:
weather_spark_df2 = spark.createDataFrame(weather_df2)

In [ ]:
weather_spark_df2.show()

+--------+------------+-----------+
|gust_mph|        name|pressure_mb|
+--------+------------+-----------+
|    17.7|    New York|     1019.0|
|     6.7| Los Angeles|     1020.0|
|    13.0|     Chicago|     1024.0|
|     7.2|     Houston|     1025.0|
|     5.6|     Phoenix|     1020.0|
|    13.4|Philadelphia|     1021.0|
|     5.7| San Antonio|     1025.0|
|     6.7|   San Diego|     1020.0|
|     6.6|      Dallas|     1027.0|
|     2.3|    San Jose|     1021.0|
|    15.5|      London|     1011.0|
|    13.4|       Paris|     1019.0|
|    13.2|      Berlin|     1023.0|
|     4.7|      Madrid|     1025.0|
|     9.4|        Rome|     1019.0|
|    12.4|       Tokyo|     1016.0|
|    17.6|     Beijing|     1018.0|
|    16.5|      Mumbai|     1008.0|
|    11.3|      Sydney|     1007.0|
|    21.5|   Cape Town|     1016.0|
+--------+------------+-----------+
only showing top 20 rows



In [42]:
weather_spark_df3 = weather_df.join(weather_spark_df2, weather_df['name'] == weather_spark_df2['name'], 'inner').select(weather_df.name,weather_df.condition,weather_df.humidity,weather_df.temperature
                                                                                                                        ,weather_df.temp_c,weather_spark_df2.gust_mph,weather_spark_df2.pressure_mb)

In [121]:
weather_spark_df3.show()

+------------+-------------+--------+-----------+------+--------+-----------+
|        name|    condition|humidity|temperature|temp_c|gust_mph|pressure_mb|
+------------+-------------+--------+-----------+------+--------+-----------+
|     Chicago|     Overcast|      67|       42.1|  5.61|    13.0|     1024.0|
|    New York|     Overcast|      41|       53.1| 11.72|    17.7|     1019.0|
| Los Angeles|Partly cloudy|      68|       61.2| 16.22|     6.7|     1020.0|
|     Houston|     Overcast|      64|       64.4|  18.0|     7.2|     1025.0|
|   San Diego|Partly cloudy|      60|       63.0| 17.22|     6.7|     1020.0|
|     Phoenix|Partly cloudy|      21|       71.1| 21.72|     5.6|     1020.0|
|Philadelphia|     Overcast|      61|       52.0| 11.11|    13.4|     1021.0|
| San Antonio|        Clear|      47|       64.9| 18.28|     5.7|     1025.0|
|      London|Partly cloudy|      81|       43.3|  6.28|    15.5|     1011.0|
|      Dallas|Partly cloudy|      69|       48.9|  9.39|     6.6

In [43]:
weather_spark_df3.groupBy('condition').agg(count('*')).show()

+------------------+--------+
|         condition|count(1)|
+------------------+--------+
|        Light rain|       3|
|             Clear|      12|
|     Partly Cloudy|       1|
|     Partly cloudy|      21|
|             Sunny|       7|
|Patchy rain nearby|       1|
|              Mist|       2|
|          Overcast|       3|
+------------------+--------+



In [44]:
weather_spark_df3 = weather_spark_df3.drop('pressure_mb')

In [45]:
weather_spark_df3 = weather_spark_df3.replace('Light rain','Drizzle','condition')

In [129]:
weather_spark_df3.orderBy('temperature').show()

+----------+-------------+--------+-----------+------+--------+
|      name|    condition|humidity|temperature|temp_c|gust_mph|
+----------+-------------+--------+-----------+------+--------+
|    Moscow|Partly cloudy|      80|       28.4|  -2.0|    16.1|
|    Warsaw|        Clear|      86|       34.2|  1.22|     8.1|
|   Beijing|        Sunny|      21|       34.5|  1.39|    17.6|
|    Vienna|Partly cloudy|      93|       35.6|   2.0|    13.0|
|    Prague|     Overcast|      93|       37.8|  3.22|     5.2|
|    Madrid|        Clear|      87|       41.7|  5.39|     4.7|
|   Chicago|     Overcast|      67|       42.1|  5.61|    13.0|
|    London|Partly cloudy|      81|       43.3|  6.28|    15.5|
|     Seoul|      Drizzle|      93|       43.5|  6.39|     5.3|
|  Helsinki|      Drizzle|      93|       44.6|   7.0|    25.6|
|   Toronto|      Drizzle|      87|       44.8|  7.11|    19.1|
| Amsterdam|Partly cloudy|      87|       44.8|  7.11|    20.0|
|      Oslo|      Drizzle|      93|     

In [138]:
def temp_range(temp):
    if temp < 50:
        return 'Low'
    elif (temp >= 50) & (temp < 70):
        return 'Medium'
    else:
        return 'High'

In [141]:
temp_range_udf = udf(temp_range,StringType())

In [142]:
weather_spark_df3.withColumn('temp_range',temp_range_udf(col('temperature'))).show()

+------------+-------------+--------+-----------+------+--------+----------+
|        name|    condition|humidity|temperature|temp_c|gust_mph|temp_range|
+------------+-------------+--------+-----------+------+--------+----------+
|     Chicago|     Overcast|      67|       42.1|  5.61|    13.0|       Low|
|    New York|     Overcast|      41|       53.1| 11.72|    17.7|    Medium|
| Los Angeles|Partly cloudy|      68|       61.2| 16.22|     6.7|    Medium|
|     Houston|     Overcast|      64|       64.4|  18.0|     7.2|    Medium|
|   San Diego|Partly cloudy|      60|       63.0| 17.22|     6.7|    Medium|
|     Phoenix|Partly cloudy|      21|       71.1| 21.72|     5.6|      High|
|Philadelphia|     Overcast|      61|       52.0| 11.11|    13.4|    Medium|
| San Antonio|        Clear|      47|       64.9| 18.28|     5.7|    Medium|
|      London|Partly cloudy|      81|       43.3|  6.28|    15.5|       Low|
|      Dallas|Partly cloudy|      69|       48.9|  9.39|     6.6|       Low|

In [50]:
weather_spark_df3.withColumn('temp_range',
                             when(col('temperature') < 50,'Low')
                             .when((col('temperature') >= 50) & (col('temperature') < 70),'Medium')
                             .otherwise('High')).show()

+------------+-------------+--------+-----------+------+--------+----------+
|        name|    condition|humidity|temperature|temp_c|gust_mph|temp_range|
+------------+-------------+--------+-----------+------+--------+----------+
| Los Angeles|        Sunny|      46|       61.0| 16.11|     6.7|    Medium|
|     Chicago|Partly cloudy|      50|       23.4| -4.78|    12.1|       Low|
|     Houston|        Sunny|      37|       64.8| 18.22|     6.7|    Medium|
|    New York|        Sunny|      39|       33.1|  0.61|    17.8|       Low|
|     Phoenix|Partly cloudy|      19|       64.0| 17.78|     3.6|    Medium|
| San Antonio|Partly cloudy|      60|       67.3| 19.61|     5.7|    Medium|
|Philadelphia|Partly cloudy|      44|       37.0|  2.78|    12.8|       Low|
|   San Diego|Partly cloudy|      37|       66.9| 19.39|     6.7|    Medium|
|      Dallas|        Sunny|      43|       55.0| 12.78|     6.9|    Medium|
|      London|Partly cloudy|      94|       54.3| 12.39|    16.7|    Medium|

In [52]:
weather_spark_df3 = weather_spark_df3.withColumn('condition',lower(weather_spark_df3.condition))

In [53]:
weather_spark_df3.show()

+------------+-------------+--------+-----------+------+--------+
|        name|    condition|humidity|temperature|temp_c|gust_mph|
+------------+-------------+--------+-----------+------+--------+
| Los Angeles|        sunny|      46|       61.0| 16.11|     6.7|
|     Chicago|partly cloudy|      50|       23.4| -4.78|    12.1|
|     Houston|        sunny|      37|       64.8| 18.22|     6.7|
|    New York|        sunny|      39|       33.1|  0.61|    17.8|
|     Phoenix|partly cloudy|      19|       64.0| 17.78|     3.6|
| San Antonio|partly cloudy|      60|       67.3| 19.61|     5.7|
|Philadelphia|partly cloudy|      44|       37.0|  2.78|    12.8|
|   San Diego|partly cloudy|      37|       66.9| 19.39|     6.7|
|      Dallas|        sunny|      43|       55.0| 12.78|     6.9|
|      London|partly cloudy|      94|       54.3| 12.39|    16.7|
|       Paris|      drizzle|      94|       50.7| 10.39|    12.3|
|    San Jose|partly cloudy|      61|       54.0| 12.22|     3.3|
|      Mad

In [54]:
window_spec = Window.partitionBy('condition').orderBy('humidity')
condition_df = weather_spark_df3.withColumn('rank',rank().over(window_spec))

In [55]:
condition_df.show()

+------------+---------+--------+-----------+------+--------+----+
|        name|condition|humidity|temperature|temp_c|gust_mph|rank|
+------------+---------+--------+-----------+------+--------+----+
|Johannesburg|    clear|      25|       75.6| 24.22|    11.9|   1|
|     Beijing|    clear|      28|       39.7|  4.28|    18.5|   2|
|        Rome|    clear|      62|       55.4|  13.0|     9.2|   3|
|     Bangkok|    clear|      65|       79.0| 26.11|     5.4|   4|
|       Tokyo|    clear|      71|       50.0|  10.0|    12.7|   5|
|      Madrid|    clear|      72|       59.5| 15.28|     7.0|   6|
|      Vienna|    clear|      75|       43.3|  6.28|    13.0|   7|
|      Sydney|    clear|      83|       68.4| 20.22|    12.7|   8|
|      Berlin|    clear|      86|       34.0|  1.11|    13.6|   9|
|      Zurich|    clear|      86|       33.8|   1.0|     5.6|   9|
|  Copenhagen|    clear|      87|       41.5|  5.28|    21.1|  11|
|   Melbourne|    clear|      88|       59.2| 15.11|     6.3| 

In [56]:
pivoted_df = weather_spark_df3.groupBy("name").pivot("condition").avg("temperature")

In [57]:
pivoted_df.show()

+------------+-----+-------+----+--------+-------------+------------------+-----+
|        name|clear|drizzle|mist|overcast|partly cloudy|patchy rain nearby|sunny|
+------------+-----+-------+----+--------+-------------+------------------+-----+
| Los Angeles| NULL|   NULL|NULL|    NULL|         NULL|              NULL| 61.0|
|     Chicago| NULL|   NULL|NULL|    NULL|         23.4|              NULL| NULL|
|     Houston| NULL|   NULL|NULL|    NULL|         NULL|              NULL| 64.8|
|    New York| NULL|   NULL|NULL|    NULL|         NULL|              NULL| 33.1|
|     Phoenix| NULL|   NULL|NULL|    NULL|         64.0|              NULL| NULL|
| San Antonio| NULL|   NULL|NULL|    NULL|         67.3|              NULL| NULL|
|Philadelphia| NULL|   NULL|NULL|    NULL|         37.0|              NULL| NULL|
|   San Diego| NULL|   NULL|NULL|    NULL|         66.9|              NULL| NULL|
|      Dallas| NULL|   NULL|NULL|    NULL|         NULL|              NULL| 55.0|
|      London| N

In [63]:
melted_df = pivoted_df.withColumn('temperature',
                                  F.explode(
                                      F.array(
                                          F.struct(F.lit('clear').alias('condition'),F.col('clear').alias('temp')),
                                          F.struct(F.lit('drizzle').alias('condition'),F.col('drizzle').alias('temp')),
                                          F.struct(F.lit('mist').alias('condition'),F.col('mist').alias('temp')),
                                          F.struct(F.lit('overcast').alias('condition'),F.col('overcast').alias('temp')),
                                          F.struct(F.lit('partly cloudy').alias('condition'),F.col('partly cloudy').alias('temp')),
                                          F.struct(F.lit('patchy rain nearby').alias('condition'),F.col('patchy rain nearby').alias('temp')),
                                          F.struct(F.lit('sunny').alias('condition'),F.col('sunny').alias('temp')),
                                      )
                                  )).select('name','temperature.condition','temperature.temp')

In [64]:
melted_df.show()

+-----------+------------------+----+
|       name|         condition|temp|
+-----------+------------------+----+
|Los Angeles|             clear|NULL|
|Los Angeles|           drizzle|NULL|
|Los Angeles|              mist|NULL|
|Los Angeles|          overcast|NULL|
|Los Angeles|     partly cloudy|NULL|
|Los Angeles|patchy rain nearby|NULL|
|Los Angeles|             sunny|61.0|
|    Chicago|             clear|NULL|
|    Chicago|           drizzle|NULL|
|    Chicago|              mist|NULL|
|    Chicago|          overcast|NULL|
|    Chicago|     partly cloudy|23.4|
|    Chicago|patchy rain nearby|NULL|
|    Chicago|             sunny|NULL|
|    Houston|             clear|NULL|
|    Houston|           drizzle|NULL|
|    Houston|              mist|NULL|
|    Houston|          overcast|NULL|
|    Houston|     partly cloudy|NULL|
|    Houston|patchy rain nearby|NULL|
+-----------+------------------+----+
only showing top 20 rows



In [65]:
melted_df_drop_null = melted_df.dropna()

In [66]:
melted_df_drop_null.show()

+------------+-------------+----+
|        name|    condition|temp|
+------------+-------------+----+
| Los Angeles|        sunny|61.0|
|     Chicago|partly cloudy|23.4|
|     Houston|        sunny|64.8|
|    New York|        sunny|33.1|
|     Phoenix|partly cloudy|64.0|
| San Antonio|partly cloudy|67.3|
|Philadelphia|partly cloudy|37.0|
|   San Diego|partly cloudy|66.9|
|      Dallas|        sunny|55.0|
|      London|partly cloudy|54.3|
|       Paris|      drizzle|50.7|
|    San Jose|partly cloudy|54.0|
|      Madrid|        clear|59.5|
|      Berlin|        clear|34.0|
|       Tokyo|        clear|50.0|
|        Rome|        clear|55.4|
|     Beijing|        clear|39.7|
|      Mumbai|     overcast|81.3|
|      Sydney|        clear|68.4|
|   Cape Town|partly cloudy|66.7|
+------------+-------------+----+
only showing top 20 rows



In [67]:
melted_df_fill_null = melted_df.fillna({'temp': 0.0})

In [68]:
melted_df_fill_null.show()

+-----------+------------------+----+
|       name|         condition|temp|
+-----------+------------------+----+
|Los Angeles|             clear| 0.0|
|Los Angeles|           drizzle| 0.0|
|Los Angeles|              mist| 0.0|
|Los Angeles|          overcast| 0.0|
|Los Angeles|     partly cloudy| 0.0|
|Los Angeles|patchy rain nearby| 0.0|
|Los Angeles|             sunny|61.0|
|    Chicago|             clear| 0.0|
|    Chicago|           drizzle| 0.0|
|    Chicago|              mist| 0.0|
|    Chicago|          overcast| 0.0|
|    Chicago|     partly cloudy|23.4|
|    Chicago|patchy rain nearby| 0.0|
|    Chicago|             sunny| 0.0|
|    Houston|             clear| 0.0|
|    Houston|           drizzle| 0.0|
|    Houston|              mist| 0.0|
|    Houston|          overcast| 0.0|
|    Houston|     partly cloudy| 0.0|
|    Houston|patchy rain nearby| 0.0|
+-----------+------------------+----+
only showing top 20 rows

